In [ ]:
!python train.py --config ../config/exp.yaml

args Namespace(model='alexnet', config='../config/exp.yaml', project='hutech_mushroom', dataset='hutech-dataset:latest')
HERE config! {'project': 'hutech_mushroom', 'dataset': 'hutech-dataset:latest', 'model': 'regnet', 'freeze': False, 'resize': 224, 'random_crop': 224, 'horizontal_flip': 0.5, 'vertical_flip': 0.5, 'random_rotation': 90, 'brightness': 0.2, 'contrast': 0.2, 'saturation': 0.2, 'normalize': 1, 'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225], 'random_erasing': 0.2, 'scheduler': 'cosine', 'es_patience': 10, 'num_classes': 4, 'lr': '1e-5', 'num_epochs': 30, 'batch_size': 4, 'optimizer': 'adam', 'weight_decay': 0.00025, 'momentum': 0.9}
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: beehappy2554 (beehappy2554-bosch-global) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.8
wandb: Run data is saved locally 